In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np

# own Modules 
from models.models import LstmMseDropout
from utils.data_loader import DataPreperatorPrediction, DataSet
from loss_module import LossModuleMse, LossModuleMle
from utils.predictor import Predictor

In [2]:
param = {
    "data" : {
        "stake_training_data" : 0.75, 
        "path" : '../../data/artifical_signals/NewBlade_degradation_freq.csv', 
    },
    "model" : {
        "path" : "../../models/MSE_model/best_model_InputSize8_LayerLstm2_HiddenLstm300_HiddenFc75_Seq150.pt",
        "input_size" : 8,
        "n_hidden_lstm" : 300,
        "sequence_size" : 50,
        "batch_size" : 1,  # Have to be 1 in prediction mode!!!
        "lstm_layer" : 2,
        "n_hidden_fc": 75,
        "dropout_rate": 0.2
    },
    "results": {
        "path_prediction" : "../visualisation/files/prediction/NewBlade_degradation_freq.csv",
        "path_residual" : "../visualisation/files/residuals/NewBlade_degradation_freq.csv"
    }
}

## Standarize Data
First we have to apply normalisation to data. That is because the model works on the representation given by its input vectors. The scale of those numbers is part of the representation.
We should apply the exact same scaling as for training data. That means storing the scale and offset used with your training data, and using that again. <br>
__The mean and variance for each feature of the training data with which the model was trained (stake: 0.75):__

### Mean and Variance from NewBlade Training Data

In [3]:
mean_training_data = [-5.37536613e-02, -2.53111489e-04, -8.82854465e+05, 7.79034183e+02, 1.45531178e+04, 1.37766733e+03, 6.50149764e-01] 
var_training_data = [1.25303578e-01, 1.16898690e-03, 2.86060835e+06, 1.64515717e+06, 6.85728371e+06, 3.63196175e+05, 8.21463343e-03]

### Mean and Variance from Artifical Training Data

In [4]:
mean_training_data= [-5.31764899e-02, -3.98576146e-04, -8.82773455e+05,  8.25672897e+02, 1.47034247e+04,  1.42685595e+03,  6.62155736e-01,  1.23172374e-02]
var_training_data = [1.28792583e-01, 1.21258617e-03, 2.90245238e+06, 1.72279458e+06, 6.83095901e+06, 3.12357562e+05, 3.89033076e-03, 5.01164766e+01]

### Mean and Variance from Artifical Training Data with First Order Difference

In [ ]:
mean_training_data=[-9.91425250e-05,  1.20717099e-05,  3.30086724e+00,  2.28118387e+00, 5.70847231e+00,  1.32484425e+00,  1.20857364e-04, -5.39472122e-04]
var_training_data =[1.04538899e-02, 1.32004086e-03, 2.77006730e+01, 6.11135598e+02, 5.13248993e+00, 3.31823108e+02, 1.11835724e-03 1.14592843e+00]

## Create DataLoader

In [5]:
data_preperator = DataPreperatorPrediction(param['data']['path'], mean_training_data, var_training_data, param["model"]["input_size"])
preprocessed_data = data_preperator.provide_data()
dataset = DataSet(preprocessed_data, timesteps=param["model"]["sequence_size"])
data_loader = DataLoader(dataset, batch_size=param['model']['batch_size'], num_workers=1, shuffle=False, drop_last=True)

## Define Model and load Parameters of trained model

In [6]:
model = LstmMseDropout(batch_size=param['model']['batch_size'], 
                       input_dim=param['model']['input_size'], 
                       n_hidden_lstm=param['model']['n_hidden_lstm'], 
                       n_hidden_fc=param['model']['n_hidden_fc'], 
                       n_layers=param['model']['lstm_layer'], 
                       dropout_rate= param['model']['dropout_rate'])

checkpoint = torch.load(param["model"]["path"])
model.load_state_dict(checkpoint['model_state_dict'])

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

## Define Loss Function

In [7]:
criterion = LossModuleMse(param["model"]["input_size"], param["model"]["batch_size"])

## Initialize Trainer

In [8]:
predictor = Predictor(model=model,
                      criterion=criterion,
                      path_data=param["data"]["path"],
                      path_results=param["results"]["path_prediction"],
                      path_residuals=param["results"]["path_residual"],
                      columns_to_ignore=["timestamp"]
                      )

## Predict

In [9]:
results = predictor.predict(data_loader)
predictor.save_prediction(results)

Start predicting


/Users/carlovoss/jupyter_notebooks/masterarbeit/venv_pm/lib/python3.6/site-packages/torch/nn/functional.py:1374: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Finished predicting


## Compute Residuals for New Blade

In [10]:
residuals = predictor.compute_residuals(results)
predictor.save_residuals(residuals)